In [ ]:
import pandas as pd
import numpy as np

In [ ]:
business = pd.read_json("yelp_academic_dataset_business.json", lines=True)

Sneak peek into 2 first rows to realise what we've got.

In [ ]:
business.head(2)

Create 2 bool arrays to finaly get only Austing AND Texas entries.

In [ ]:
austin_tx_bool = np.array(business.city == 'Austin') * np.array(business.state == 'TX')
austin_tx = business[austin_tx_bool]

In [ ]:
# austin_tx.to_json('austin_tx.json')

In [ ]:
austin_tx = business[austin_tx_bool].drop(['city', 'state', 'business_id', 'address'], axis=1)
austin_tx.shape

In [ ]:
austin_tx.isna().sum()

In [ ]:
austin_tx.dropna(inplace=True)
austin_tx.shape
# austin_tx.to_json('austin_tx_nona.json')


In [ ]:
austin_tx.head(3)

In [ ]:
attribs = dict(austin_tx[austin_tx.name == "Lane Wells Jewelry Repair"].attributes)

In [ ]:
austin_tx.attributes[8]

In [ ]:
austin_tx.categories[8]